<a href="https://www.kaggle.com/code/rochellegonzalez/fifa21-data-cleaning?scriptVersionId=160925334" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import decimal
import os

In [2]:
fifa21 = pd.read_csv('/kaggle/input/fifa-21-messy-raw-dataset-for-cleaning-exploring/fifa21_raw_data.csv')

/tmp/ipykernel_18/2869911767.py:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa21 = pd.read_csv('/kaggle/input/fifa-21-messy-raw-dataset-for-cleaning-exploring/fifa21_raw_data.csv')


## Dataset Overview

1. **head** : There are 5 rows and 77 columns
2. **info** : dtypes: int64(55), object(22)
RangeIndex: 18979 entries, 0 to 18978

In [3]:
fifa21.head()

,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,https://cdn.sofifa.com/players/158/023/21_60.png,Lionel Messi,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,RW ST CF,L. Messi,33,93,93,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n,...,Medium,Low,5 ★,85,92,91,95,38,65,\n372
1,https://cdn.sofifa.com/players/020/801/21_60.png,C. Ronaldo dos Santos Aveiro,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,ST LW,Cristiano Ronaldo,35,92,92,\n\n\n\nJuventus\n2018 ~ 2022\n\n,...,High,Low,5 ★,89,93,81,89,35,77,\n344
2,https://cdn.sofifa.com/players/200/389/21_60.png,Jan Oblak,http://sofifa.com/player/200389/jan-oblak/210005/,Slovenia,GK,J. Oblak,27,91,93,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n,...,Medium,Medium,3 ★,87,92,78,90,52,90,\n86
3,https://cdn.sofifa.com/players/192/985/21_60.png,Kevin De Bruyne,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,CAM CM,K. De Bruyne,29,91,91,\n\n\n\nManchester City\n2015 ~ 2023\n\n,...,High,High,4 ★,76,86,93,88,64,78,\n163
4,https://cdn.sofifa.com/players/190/871/21_60.png,Neymar da Silva Santos Jr.,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,LW CAM,Neymar Jr,28,91,91,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n,...,High,Medium,5 ★,91,85,86,94,36,59,\n273


In [4]:
fifa21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   photoUrl          18979 non-null  object
 1   LongName          18979 non-null  object
 2   playerUrl         18979 non-null  object
 3   Nationality       18979 non-null  object
 4   Positions         18979 non-null  object
 5   Name              18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Team & Contract   18979 non-null  object
 10  ID                18979 non-null  int64 
 11  Height            18979 non-null  object
 12  Weight            18979 non-null  object
 13  foot              18979 non-null  object
 14  BOV               18979 non-null  int64 
 15  BP                18979 non-null  object
 16  Growth            18979 non-null  int64 
 17  Joined      

***determine the metric conversion by printing the data types of height and weight***

In [5]:
print(fifa21.dtypes[['Height','Weight']])
print(fifa21[['Height', 'Weight']])

Height    object
Weight    object
dtype: object
      Height  Weight
0       5'7"  159lbs
1       6'2"  183lbs
2       6'2"  192lbs
3      5'11"  154lbs
4       5'9"  150lbs
...      ...     ...
18974  5'10"  154lbs
18975   5'7"  128lbs
18976  5'10"  146lbs
18977   5'9"  143lbs
18978   5'9"  150lbs

[18979 rows x 2 columns]


***convert height and weight into str 
Convert Height column from feet/inches to cm
convert Weight column from lbs to kg***

In [6]:
def hwConvert(x):

    if x.endswith('lbs') == True:
        x = x.replace("lbs","")
        a = str(round(int(x)*.453592))
        return a + 'kg'
    else:
        x = x.replace("\'", " ").replace("\"", "").split()
        a = int(x[0])*12 + int(x[1])
        fin = str(round(a*2.54)) + 'cm'
        return fin

fifa21['Weight'] = fifa21['Weight'].map(hwConvert)
fifa21['Height'] = fifa21['Height'].map(hwConvert)
fifa21[['Height','Weight']]

,Height,Weight
0,170cm,72kg
1,188cm,83kg
2,188cm,87kg
3,180cm,70kg
4,175cm,68kg
...,...,...
18974,178cm,70kg
18975,170cm,58kg
18976,178cm,66kg
18977,175cm,65kg


***Split join data into separate months and year columns***

In [7]:
fifa21['Joined'] = pd.to_datetime(fifa21.Joined)

In [8]:
fifa21['Joined_Year'] = fifa21.Joined.dt.year
fifa21['Joined_Month'] = fifa21.Joined.dt.month
fifa21['Joined_Day'] = fifa21.Joined.dt.day

***format Team and Contract columns***
1. remove the unnecessary characters
2. split team and contract columns

In [9]:
fifa21['Team & Contract'] = fifa21['Team & Contract'].str.lstrip('\n').str.rstrip('\n')
fifa21[['Team','Contract']] = fifa21['Team & Contract'].str.split('\n', expand=True)

## Formatting and Removing Unnecessary Characters in the Columns

1. Hits
2. IR
3. SM

In [10]:
fifa21['Hits'] = fifa21['Hits'].str.lstrip('\n').str.rstrip('\n')

In [11]:
fifa21[["IR"]]
fifa21[["SM"]]
#fifa21[["WF"]]

,SM
0,4★
1,5★
2,1★
3,4★
4,5★
...,...
18974,2★
18975,2★
18976,2★
18977,2★


***Remove unnecessary characters in the IR, and SM column***

In [12]:
fifa21['IR'] = fifa21['IR'].str.lstrip('★').str.rstrip('★')
fifa21['SM'] = fifa21['SM'].str.lstrip('★').str.rstrip('★')
fifa21[["IR","SM"]]

,IR,SM
0,5,4
1,5,5
2,3,1
3,4,4
4,5,5
...,...,...
18974,1,2
18975,1,2
18976,1,2
18977,1,2


***Creating a general function that can be applied to Value, Wage and Release Clause***

In [13]:
fifa21[['Wage', 'Value','Release Clause']]

,Wage,Value,Release Clause
0,€560K,€67.5M,€138.4M
1,€220K,€46M,€75.9M
2,€125K,€75M,€159.4M
3,€370K,€87M,€161M
4,€270K,€90M,€166.5M
...,...,...,...
18974,€1K,€35K,€57K
18975,€500,€60K,€165K
18976,€1K,€40K,€70K
18977,€500,€60K,€165K


In [14]:
def value_setting(value): 
    if isinstance(value, str):
        if value.find("M") != -1:
            return int(float(value.replace("M", "").replace('€', '')) * 1000000)
        elif value.find("K") != -1: 
            return int(value.replace("K", "").replace("€", "")) * 1000 
    else:
        return int(value)

In [15]:
fifa21['Value'] = fifa21['Value'].apply(value_setting)
fifa21['Wage'] = fifa21['Wage'].apply(value_setting)
fifa21['Release Clause'] = fifa21['Release Clause'].apply(value_setting)

In [16]:
fifa21[['Wage', 'Value','Release Clause']]

,Wage,Value,Release Clause
0,560000.0,67500000.0,138400000.0
1,220000.0,46000000.0,75900000.0
2,125000.0,75000000.0,159400000.0
3,370000.0,87000000.0,161000000.0
4,270000.0,90000000.0,166500000.0
...,...,...,...
18974,1000.0,35000.0,57000.0
18975,NaN,60000.0,165000.0
18976,1000.0,40000.0,70000.0
18977,NaN,60000.0,165000.0


In [17]:
fifa21[['Positions']]

,Positions
0,RW ST CF
1,ST LW
2,GK
3,CAM CM
4,LW CAM
...,...
18974,CB
18975,ST
18976,CB
18977,CM


***Split positions***

In [18]:
fifa21[["Position 1", "Position 2", "Position 3"]] = fifa21["Positions"].str.split(" ", expand=True)

In [19]:
temp_position = []
for x in range(len(fifa21['Positions'])):
    y = sorted(fifa21['Positions'][x].split(" "))
    yx = ' '.join(y)
    temp_position.append(yx)

fifa21['Positions'] = temp_position
fifa21['Positions'] = fifa21['Positions'].astype(object)
fifa21['Positions'].unique()

array(['CF RW ST', 'LW ST', 'GK', 'CAM CM', 'CAM LW', 'ST', 'LW RW ST',
       'RW', 'LW', 'CB', 'CDM', 'CF ST', 'CDM CM', 'CDM RB', 'CAM CF',
       'LW RW', 'CM', 'CAM LM RM', 'RB', 'CAM CM RW', 'LB', 'CF', 'CF LM',
       'CF LW ST', 'CAM RM RW', 'CAM CF ST', 'CAM CF RM', 'LM ST',
       'CAM CDM CM', 'LM', 'CAM RW', 'CB CDM', 'LM RM RW', 'RM RW',
       'CAM CF LW', 'CAM CF CM', 'CF LW', 'CM LM RM', 'LB LM',
       'CAM CM RM', 'LM LW RM', 'CM LB LM', 'CM RM', 'CAM CM LW', 'CB LB',
       'RB RM', 'RW ST', 'LM LW RW', 'LB RB', 'RM', 'CAM RM', 'RB RWB',
       'CB CDM CM', 'CAM RM ST', 'LM RM', 'LM LWB', 'CF RM ST',
       'CF RM RW', 'CF LM RM', 'CF RM', 'CM RB', 'CAM LW RW', 'CAM CM LM',
       'CM RM ST', 'CDM CM RB', 'CAM ST', 'RM ST', 'CB LB LWB',
       'CAM LW ST', 'RM RWB', 'CB CDM LB', 'LM LW', 'LW RM RW', 'CM LM',
       'LB LM RM', 'LW RM ST', 'CAM CM ST', 'LB LWB', 'CB RB', 'RM RW ST',
       'LWB', 'CAM LM', 'CAM', 'RB RM RW', 'CF LM LW', 'CAM LM LW',
       'CB RB RWB'

## Dropping Unnecessary Columns or Duplicate Columns Resulting from Splitting the Columns

**Unnecessary Columns**

1. PhotoURL
2. Player URl

**Duplicate Columns**
1. Joined
2. Team and Contract

In [20]:
fifa21 = fifa21.drop(["photoUrl"], axis =1)
fifa21 = fifa21.drop(["playerUrl"], axis =1)

In [21]:
fifa21 = fifa21.drop(["Joined"], axis=1)
fifa21 = fifa21.drop(["Team & Contract"], axis=1)
fifa21 = fifa21.drop("Positions", axis=1)

In [22]:
fifa21.head()

,LongName,Nationality,Name,Age,↓OVA,POT,ID,Height,Weight,foot,...,PHY,Hits,Joined_Year,Joined_Month,Joined_Day,Team,Contract,Position 1,Position 2,Position 3
0,Lionel Messi,Argentina,L. Messi,33,93,93,158023,170cm,72kg,Left,...,65,372,2004,7,1,FC Barcelona,2004 ~ 2021,RW,ST,CF
1,C. Ronaldo dos Santos Aveiro,Portugal,Cristiano Ronaldo,35,92,92,20801,188cm,83kg,Right,...,77,344,2018,7,10,Juventus,2018 ~ 2022,ST,LW,None
2,Jan Oblak,Slovenia,J. Oblak,27,91,93,200389,188cm,87kg,Right,...,90,86,2014,7,16,Atlético Madrid,2014 ~ 2023,GK,None,None
3,Kevin De Bruyne,Belgium,K. De Bruyne,29,91,91,192985,180cm,70kg,Right,...,78,163,2015,8,30,Manchester City,2015 ~ 2023,CAM,CM,None
4,Neymar da Silva Santos Jr.,Brazil,Neymar Jr,28,91,91,190871,175cm,68kg,Right,...,59,273,2017,8,3,Paris Saint-Germain,2017 ~ 2022,LW,CAM,None


In [23]:
fifa21.to_csv("cleaned_fifa21_data.csv", index = False)